In [14]:
# import pprint
# from zipfile import ZipFile

# path = "./PythonTools-3.7.0-py2.py3-none-any.whl"
# names = ZipFile(path).namelist()
# pprint.pprint(names)

In [15]:
from pathlib import Path
import json

import pydicom as dicom
import numpy as np
from PIL import Image

import matplotlib.pyplot as plt
import matplotlib.animation as animation

import slicerio # https://pypi.org/project/slicerio/

from PythonTools.rek2py import rek2py
# from PythonTools.raw2dicom import raw2dicom

In [16]:
path_variable = "/mnt/d/Datasets/shoes/"
save_path = path_variable + "images_3d/"

In [17]:
N_CONVS = 2


MODELNAME = f"SegFormer_{N_CONVS}conv"
# MODELNAME = "SegFormer"
# SHVIT_SEGFORMER = False

NUM_LAYERS = 10


DEPTH, HEIGHT, WIDTH =  32, 32, 32
# DEPTH, HEIGHT, WIDTH =  64,  64,  64
# DEPTH, HEIGHT, WIDTH =  80,  80,  80
# DEPTH, HEIGHT, WIDTH =  96,  96,  96
# DEPTH, HEIGHT, WIDTH = 112, 112, 112 
# DEPTH, HEIGHT, WIDTH = 128, 128, 128 
# DEPTH, HEIGHT, WIDTH = 160, 160, 160 
# DEPTH, HEIGHT, WIDTH = 192, 192, 192 
# DEPTH, HEIGHT, WIDTH = 196, 196, 196
# DEPTH, HEIGHT, WIDTH = 224, 224, 224
# DEPTH, HEIGHT, WIDTH = 240, 240, 240
# DEPTH, HEIGHT, WIDTH = 256, 256, 256
# DEPTH, HEIGHT, WIDTH = 272, 272, 272
# DEPTH, HEIGHT, WIDTH = 288, 288, 288
# DEPTH, HEIGHT, WIDTH = 320, 320, 320
# DEPTH, HEIGHT, WIDTH = 336, 336, 336
# DEPTH, HEIGHT, WIDTH = 352, 352, 352
# DEPTH, HEIGHT, WIDTH = 448, 448, 448
# DEPTH, HEIGHT, WIDTH = 464, 464, 464
# DEPTH, HEIGHT, WIDTH = 480, 480, 480
# DEPTH, HEIGHT, WIDTH = 512, 512, 512
# DEPTH, HEIGHT, WIDTH = 896, 896, 896

factor = 2**N_CONVS

In [18]:
def list_all_files(directory, ext):

    directory_path = Path(directory)
    # List all files in the directory and subdirectories
    all_files = list(directory_path.rglob(ext))  

    # Filter out directories
    files = [file for file in all_files if file.is_file()]
    return files


# Example usage
directory = path_variable + "_volumes/"
files = list_all_files(directory, "*.rek")
files

[PosixPath('/mnt/d/Datasets/shoes/_volumes/Adidas_Martin-weiss.rek'),
 PosixPath('/mnt/d/Datasets/shoes/_volumes/Adidas_Martin.rek'),
 PosixPath('/mnt/d/Datasets/shoes/_volumes/Bruschi_down2_2_2.rek'),
 PosixPath('/mnt/d/Datasets/shoes/_volumes/Camper_down2_2_2.rek'),
 PosixPath('/mnt/d/Datasets/shoes/_volumes/Citywalk-sit-taupe-34_down2_2_2.rek'),
 PosixPath('/mnt/d/Datasets/shoes/_volumes/Citywalk-sit-taupe-36_down2_2_2.rek'),
 PosixPath('/mnt/d/Datasets/shoes/_volumes/Citywalk-sit-taupe-39_down2_2_2.rek'),
 PosixPath('/mnt/d/Datasets/shoes/_volumes/Citywalk-sit-taupe-40_down2_2_2.rek'),
 PosixPath('/mnt/d/Datasets/shoes/_volumes/Citywalk-sit-taupe-42_down2_2_2.rek'),
 PosixPath('/mnt/d/Datasets/shoes/_volumes/Herrenschuh_43p5_down2_2_2.rek'),
 PosixPath('/mnt/d/Datasets/shoes/_volumes/Lidl_43_down2_2_2.rek'),
 PosixPath('/mnt/d/Datasets/shoes/_volumes/Lloyd_38_down2_2_2.rek'),
 PosixPath('/mnt/d/Datasets/shoes/_volumes/Lloyd_pink_down2_2_2.rek'),
 PosixPath('/mnt/d/Datasets/shoes/_v

In [19]:
# after run check how shoes are oriented in the box; repeat only the files where shoes are not displayed from the side - modify transpose
rotate_shoes = [path_variable + "_volumes/Adidas_Martin.rek",path_variable + "_volumes/Citywalk-sit-taupe-34_down2_2_2.rek", 
                path_variable + "_volumes/Citywalk-sit-taupe-36_down2_2_2.rek", path_variable + "_volumes/Citywalk-sit-taupe-40_down2_2_2.rek", 
                path_variable + "_volumes/Schuh_Martin_down2_2_2.rek", path_variable + "_volumes/Sneaker_Dana_Nike.rek",
                path_variable + "_volumes/Sneaker_Dana_Puma_Flyer.rek"]

# Convert to pathlib.Path objects
rotate_shoes = [Path(folder) for folder in rotate_shoes]
rotate_shoes

[PosixPath('/mnt/d/Datasets/shoes/_volumes/Adidas_Martin.rek'),
 PosixPath('/mnt/d/Datasets/shoes/_volumes/Citywalk-sit-taupe-34_down2_2_2.rek'),
 PosixPath('/mnt/d/Datasets/shoes/_volumes/Citywalk-sit-taupe-36_down2_2_2.rek'),
 PosixPath('/mnt/d/Datasets/shoes/_volumes/Citywalk-sit-taupe-40_down2_2_2.rek'),
 PosixPath('/mnt/d/Datasets/shoes/_volumes/Schuh_Martin_down2_2_2.rek'),
 PosixPath('/mnt/d/Datasets/shoes/_volumes/Sneaker_Dana_Nike.rek'),
 PosixPath('/mnt/d/Datasets/shoes/_volumes/Sneaker_Dana_Puma_Flyer.rek')]

In [20]:
from scipy import ndimage

def resize_volume(volume, depth=DEPTH, height=HEIGHT, width=WIDTH, factor=1):
    
    (vol_depth, vol_height, vol_width) = volume.shape
    
    depth_factor = depth/vol_depth/factor
    height_factor = height/vol_height/factor
    width_factor = width/vol_width/factor
    
    print("xx"*20, volume.shape)
    print("xx"*20, depth_factor, height_factor, width_factor)
    
    volume = ndimage.zoom(volume, (depth_factor, height_factor, width_factor), order=1)
    print("xx"*20, volume.shape)
    
    return volume

In [21]:
def saveVolumes(folder, files):
    for file in files:
        
        # Get the file name without extension
        file_path  = Path(file)
        filename =  file_path.stem
        print(filename)

        # dicomdata = raw2dicom(image=volume, ezrt_header=header)
        header, volume = rek2py(filepath=file, switch_order=True)
        print(volume.shape)
        
        _min, _max = 1, 99
        if "PetrolioSch-40-float_down2_2_2" in str(file):
            _min, _max = 4, 99            
        
        values = volume.flatten()
        vmin = np.percentile(values, _min)
        vmax = np.percentile(values, _max)
        
        scale = (volume.astype(np.float64) - vmin)/(vmax-vmin + 1e-5)
        scale = np.clip(scale, 0, 1).astype(np.float32)
    
        # createAnimation(filename, volume)
        if file in rotate_shoes:
            orientation = (1,2,0)
        else:
            orientation = (2,1,0)
            
        volume = scale.transpose(orientation)      
        volume = resize_volume(volume)
        
        folder_path = Path(folder)
        folder_path.mkdir(parents=True, exist_ok=True)
        file_path = f"{folder_path.joinpath(filename)}" #.npz"
        # np.savez_compressed(file_path, volume)
        np.save(file_path, volume)
    

target_directory = f"{save_path}images_3d_{MODELNAME}_{DEPTH}x{HEIGHT}x{WIDTH}/volumes"
saveVolumes(target_directory, files)

Adidas_Martin-weiss
(993, 988, 988)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (988, 988, 993)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx 0.032388663967611336 0.032388663967611336 0.032225579053373615
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (32, 32, 32)
Adidas_Martin
(993, 990, 990)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (990, 990, 993)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx 0.03232323232323232 0.03232323232323232 0.032225579053373615
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (32, 32, 32)
Bruschi_down2_2_2
(876, 650, 475)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (475, 650, 876)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx 0.06736842105263158 0.04923076923076923 0.0365296803652968
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (32, 32, 32)
Camper_down2_2_2
(880, 675, 425)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (425, 675, 880)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx 0.07529411764705882 0.047407407407407405 0.03636363636363636
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (32, 32, 32)
Citywalk-sit-ta

In [22]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "vscode"


def plotVolume(folder, file):
    
    # Get the file name without extension
    file_path  = Path(file)
    filename =  file_path.stem
    print(filename)    
    
    # Load 3D numpy array
    shoe_volume = np.load(f"{save_path}images_3d_{MODELNAME}_{DEPTH}x{HEIGHT}x{WIDTH}/volumes/{filename}.npy")  # Ensure this is a (128,128,128) NumPy array

    # Create the 3D grid
    x, y, z = np.mgrid[:DEPTH, :HEIGHT, :WIDTH]

    # Create the volume plot
    fig = go.Figure(data=go.Volume(
        x=x.flatten(),
        y=y.flatten(),
        z=z.flatten(),
        value=shoe_volume.flatten().astype(np.float16),
        opacity=0.1,  # Adjust for visibility
        surface_count=30,  # Number of isosurfaces
        colorscale="gray"  # Set grayscale colormap
    ))

    # Show the figure
    fig.update_layout(
        width=1000,  # Set the width in pixels
        height=1000,   # Set the height in pixels
        scene=dict(
            xaxis=dict(showbackground=False),
            yaxis=dict(showbackground=False),
            zaxis=dict(showbackground=False),
        )
    )

    filename = f"plot_{DEPTH}x{HEIGHT}x{WIDTH}_{filename}.html"
    folder_path = Path(folder)
    folder_path.mkdir(parents=True, exist_ok=True)
    file_path = f"{folder_path.joinpath(filename)}"

    print(file_path)
    fig.write_html(file_path)
    
    
plot_directory = f"{save_path}plots"
volume_directory = f"{save_path}images_3d_{MODELNAME}_{DEPTH}x{HEIGHT}x{WIDTH}/volumes"
files = list_all_files(volume_directory, "*.npy")
    
# for file in files[:5]:
#     plotVolume(plot_directory, file)

In [23]:
directory = path_variable + "annotations/"
files = list_all_files(directory, "*.seg.nrrd")

for file in files:
    
    # Get the file name without extension
    file_path  = Path(file)
    filename =  file_path.stem
    print(filename)    

    segmentation = slicerio.read_segmentation(str(file), skip_voxels=True)

    number_of_segments = len(segmentation["segments"])
    print(f"Number of segments: {number_of_segments}")

    segment_names = slicerio.segment_names(segmentation)
    print(f"Segment names: {', '.join(segment_names)}")

## Extract image and masks for layer xyz of all shoes

In [24]:
rotate_shoes = [path_variable + "_annotations/Adidas_Martin.seg.nrrd", path_variable + "_annotations/Citywalk 34.seg.nrrd", 
                path_variable + "_annotations/Citywalk 36.seg.nrrd", path_variable + "_annotations/Citywalk 40.seg.nrrd", 
                path_variable + "_annotations/Schuh_Martin_down2_2_2.seg.nrrd", path_variable + "_annotations/Sneaker_Dana_Nike.seg.nrrd", 
                path_variable + "_annotations/Sneaker_Dana_Puma_Flyer.seg.nrrd"]

rotate_shoes = [Path(folder) for folder in rotate_shoes]

class_dict = {"Karton": 1, "Außensohle": 2, "Innensohle": 3, "Obermaterial": 4, "Zunge": 5, "Füllmaterial": 6}

In [25]:
key = "Zunge"
class_dict[key]

5

In [26]:
def createMask(folder, file):

    segmentation_info = slicerio.read_segmentation(file)
    segment_names = slicerio.segment_names(segmentation_info) # Get semgnet_names where 3D Slicer

    segment_names_to_labels = []
    for j, sn in enumerate(segment_names):
        segment_names_to_labels.append((sn, j+1)) 
        if not segment_names_to_labels:
            continue
        
    # Numbering according to Segmented labels of 3D Slicer
    data_Karton         = slicerio.extract_segments(segmentation_info, [[item for item in segment_names_to_labels if item[0] == "Karton"][0]])
    data_Aussensohle    = slicerio.extract_segments(segmentation_info, [[item for item in segment_names_to_labels if item[0] == "Auensohle"][0]])
    data_Innensohle     = slicerio.extract_segments(segmentation_info, [[item for item in segment_names_to_labels if item[0] == "Innensohle"][0]])
    data_Obermaterial   = slicerio.extract_segments(segmentation_info, [[item for item in segment_names_to_labels if item[0] == "Obermaterial"][0]])
    data_Zunge          = slicerio.extract_segments(segmentation_info, [[item for item in segment_names_to_labels if item[0] == "Zunge"][0]])
    data_Fuellmaterial  = slicerio.extract_segments(segmentation_info, [[item for item in segment_names_to_labels if item[0] == "Fllmaterial"][0]])

    if file in rotate_shoes:
        orientation = (1,0,2) # these shoes need different orientation / transpose
    else:
        orientation = (0,1,2) # normal orientation for all shoes
            
    print(orientation)
    data_Karton        = data_Karton.get("voxels").transpose(orientation)            
    data_Aussensohle   = data_Aussensohle.get("voxels").transpose(orientation)            
    data_Obermaterial  = data_Obermaterial.get("voxels").transpose(orientation)            
    data_Innensohle    = data_Innensohle.get("voxels").transpose(orientation)            
    data_Zunge         = data_Zunge.get("voxels").transpose(orientation)            
    data_Fuellmaterial = data_Fuellmaterial.get("voxels").transpose(orientation)            
        
    # print(np.unique(data_Karton))
    # print(np.unique(data_Aussensohle))
    # print(np.unique(data_Obermaterial))
    # print(np.unique(data_Innensohle))
    # print(np.unique(data_Zunge))
    # print(np.unique(data_Fuellmaterial))
    
    data_Karton = resize_volume(data_Karton, factor=factor)
    data_Aussensohle = resize_volume(data_Aussensohle, factor=factor)
    data_Obermaterial = resize_volume(data_Obermaterial, factor=factor)
    data_Innensohle = resize_volume(data_Innensohle, factor=factor)
    data_Zunge = resize_volume(data_Zunge, factor=factor)
    data_Fuellmaterial = resize_volume(data_Fuellmaterial, factor=factor)


    # Create an empty volume filled with zeros
    final_volume = np.zeros(data_Karton.shape, dtype=np.uint8)
    final_volume[data_Karton>0] = class_dict["Karton"]
    final_volume[data_Aussensohle>0] = class_dict["Außensohle"]
    final_volume[data_Innensohle>0] = class_dict["Innensohle"]
    final_volume[data_Obermaterial>0] = class_dict["Obermaterial"]
    final_volume[data_Zunge>0] = class_dict["Zunge"]
    final_volume[data_Fuellmaterial>0] = class_dict["Füllmaterial"]

    print(final_volume.shape, np.unique(final_volume))
    
    
    if file.suffix:  # Check again if there's at least one suffix
        filename =  file.stem

        if file.suffix:  # Check again if there's at least one suffix
            filename = filename.rsplit('.', 1)[0]  # Remove the last part after the last '.'
    print(filename)  
    
    folder_path = Path(folder)
    folder_path.mkdir(parents=True, exist_ok=True)
    file_path = f"{folder_path.joinpath(filename)}" #.npz"
    # np.savez_compressed(file_path, volume)
    np.save(file_path, final_volume)

        
    
    
# extract mask information  
target_directory = f"{save_path}images_3d_{MODELNAME}_{DEPTH}x{HEIGHT}x{WIDTH}/masks"
annotation_directory = path_variable + "_annotations/"

files = list_all_files(annotation_directory, "*.seg.nrrd")
for file in files:
    print(file)
    createMask(target_directory, file)   

/mnt/d/Datasets/shoes/_annotations/Adidas_Martin-weiss.seg.nrrd
(0, 1, 2)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (988, 988, 993)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx 0.008097165991902834 0.008097165991902834 0.008056394763343404
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (8, 8, 8)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (988, 988, 993)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx 0.008097165991902834 0.008097165991902834 0.008056394763343404
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (8, 8, 8)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (988, 988, 993)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx 0.008097165991902834 0.008097165991902834 0.008056394763343404
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (8, 8, 8)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (988, 988, 993)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx 0.008097165991902834 0.008097165991902834 0.008056394763343404
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (8, 8, 8)
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx (988, 988, 993)
xxxxxxxxxxxxxxxxxxxxx